In [ ]:
import pandas as pd
import pyam
import nomenclature

In [ ]:
iamc_args = dict(
    model="State of CDR (2024) - Voluntary Carbon Market",
    scenario="Observed Data",
)

In [ ]:
dsd = nomenclature.DataStructureDefinition("../definitions/")

In [ ]:
df_list = list()

## Conventional Credits

In [ ]:
for sheet_name, _type in (("17(a)- Conventional Issuances", "Issued"), ("17(b)-Conventional Retirements", "Sold")):
    data = pd.read_excel(
        "source/4_Voluntary_Carbon_Market/C4- Updated Full Dashboard & Summary Data.xlsx",
        sheet_name=sheet_name,
        header=0,
        nrows=5,
        usecols="A:C",
    )
    data.columns = ["variable", 2022, 2023]
    
    df_method = pyam.IamDataFrame(
        data,
        **iamc_args,
        unit="t CO2",
        region="World",
    )
    df_method.rename(
        variable={
            "Afforestation/Reforestation": "Afforestation and Reforestation",
            "Wetland Restoration": "Peatland and Wetland Restoration",
            "Wooden Building Elements": "Durable Wood Products",
        },
        inplace=True,
    )
    df_method.rename(
        variable=dict([(i, f"Carbon Credits|Conventional CDR|Tons {_type}|{i}") for i in df_method.variable]),
        inplace=True,
    )
    df_method.aggregate(f"Carbon Credits|Conventional CDR|Tons {_type}", append=True)

    data = pd.read_excel(
        "source/4_Voluntary_Carbon_Market/C4- Updated Full Dashboard & Summary Data.xlsx",
        sheet_name=sheet_name,
        header=9,
        nrows=37,
        usecols="A:C",
    )
    data.columns = ["region", 2022, 2023]
    
    df_country = pyam.IamDataFrame(
        data, 
        **iamc_args,
        variable=f"Carbon Credits|Conventional CDR|Tons {_type}",
        unit="t CO2",
    )

    df_list.append(pyam.concat([df_method, df_country]))

## Novel credits

In [ ]:
for sheet_name, _type in (("18(a)-Novel Purchases", "Sold"), ("18(b)- Novel Deliveries", "Delivered")):
    data = pd.read_excel(
        "source/4_Voluntary_Carbon_Market/C4- Updated Full Dashboard & Summary Data.xlsx",
        sheet_name=sheet_name,
        header=0,
        nrows=11,
        usecols="A:C",
    )
    data.columns = ["variable", 2022, 2023]
    
    df_method = pyam.IamDataFrame(
        data,
        **iamc_args,
        unit="t CO2",
        region="World",
    )
    df_method.rename(
        variable={
            "BECCS": "Bioenergy with CCS",
            "Biomass Removal": "Biomass Burial",
            "Biooil": "Bio-Oil Storage",
            "DAC": "Direct Air Capture with CCS",
            "Direct Ocean Removal": "Direct Ocean Capture",
            "Microalgae": "Biomass Sinking",
            "Not disclosed": "Not Disclosed",
        },
        inplace=True,
    )
    df_method.rename(
        variable=dict([(i, f"Carbon Credits|Novel CDR|Tons {_type}|{i}") for i in df_method.variable]),
        inplace=True,
    )
    df_method.aggregate(f"Carbon Credits|Novel CDR|Tons {_type}", append=True)

    df_list.append(pyam.concat([df_method]))

## Number of active projects

In [ ]:
data = pd.read_excel(
    "source/4_Voluntary_Carbon_Market/C4- Updated Full Dashboard & Summary Data.xlsx",
    sheet_name="Full Dashboard",
)

In [ ]:
active_projects_list = []

for year in [2022, 2023]:
    columns = [i for i in data.columns if str(year) in i]
    _df = data[data[columns].T.sum() > 0].groupby(["Region", "SoCDR Classification"]).count()["Project ID"].reset_index()
    _df["year"] = year

    active_projects_list.append(
         pyam.IamDataFrame(
            _df,
            **iamc_args,
            unit="",
            variable="SoCDR Classification",
            value="Project ID",
        ).filter(variable=["Avoided Emissions", "Emission Reduction"], keep=False)
    )

df_active_project = pyam.concat(active_projects_list)

In [ ]:
prefix = "Carbon Credits|Active Projects"
df_active_project.rename(
    variable=dict(
        [(i, f"{prefix}|{i}") for i in df_active_project.variable]
    ),
    inplace=True
)
df_active_project.aggregate(prefix, append=True)

In [ ]:
df_active_project.aggregate(prefix, append=True)

In [ ]:
df_active_project.aggregate_region(df_active_project.variable, append=True)

In [ ]:
df_active_project.filter(year=2022).plot.bar(x="region", bars="variable", stacked=True, )

In [ ]:
df_active_project.filter(year=2023).plot.bar(x="region", bars="variable", stacked=True, )

In [ ]:
df_list.append(df_active_project)

## Merge data

In [ ]:
df = pyam.concat(df_list)

In [ ]:
df.rename(region={"Republic of Congo": "Congo"}, inplace=True)

In [ ]:
df.convert_unit("t CO2", "Mt CO2", inplace=True)

In [ ]:
dsd.validate(df)

In [ ]:
df.filter(region="World", variable="Carbon Credits|Conventional CDR|Tons Sold|*").plot.bar(stacked=True)

In [ ]:
dsd.validate(df_active_project)

# Import to ixmp4 database

In [ ]:
import ixmp4

In [ ]:
platform = ixmp4.Platform("socdr-dev")

In [ ]:
run = platform.runs.get(**iamc_args)

In [ ]:
run.iamc.remove(run.iamc.tabulate())

In [ ]:
run.iamc.add(df.data)